# 00 — Retrieval Debug Sandbox

Этот ноутбук — песочница для отладки ретрива:

- проверить, как работают BM25 / dense / hybrid,
- подобрать `k`, `alpha`, MMR и чанкеры,
- глазами посмотреть, **что именно** возвращает ретривер для конкретных запросов.

Идея: сначала подружиться с корпусом и ретривом, а уже потом строить полноценный RAG-пайплайн.


In [ ]:
# === Импорты основных библиотек ===
import os
from typing import List, Dict, Any, Optional

import numpy as np

# Если хочешь использовать pandas для красивых табличек
import pandas as pd

# === Импорты из нашего RAG-пакета ===
from rag.corpus import build_corpus_from_texts, Corpus
from rag.chunkers import paragraph_chunker, char_chunker, token_chunker
from rag.indices import build_bm25_index, build_dense_index, BM25Index, DenseIndex
from rag.retrieval import (
    bm25_candidates,
    dense_candidates,
    hybrid_candidates,
)
from rag.candidates import Candidate
from rag.debug import (
    summarize_candidates,
    analyze_stage_transition,  # если понадобится смотреть до/после CE/MMR
)

# === (Опционально) эмбеддинг-модель ===
# Тут оставляю TODO: под конкретную задачу подставишь нужную модель.
# Частый вариант: sentence-transformers или huggingface transformers.

# from sentence_transformers import SentenceTransformer
# EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"


# === Пути к данным (TODO: поменяй под задачу) ===
DATA_DIR = "../data"   # корень с данными задачи
CORPUS_PATH = os.path.join(DATA_DIR, "corpus.jsonl")     # или .csv / что дадут
QUESTIONS_PATH = os.path.join(DATA_DIR, "questions.jsonl")  # если есть вопросы

# === Базовые параметры ===

# Тип чанкера (можно переопределить ниже)
DEFAULT_CHUNKER = paragraph_chunker

# Сколько top-k документов смотреть глазами
TOP_K_DEBUG = 5

# Фиксируем seed, чтобы результаты были воспроизводимы
np.random.seed(42)


In [ ]:
import json

def load_corpus_jsonl(path: str) -> List[Dict[str, Any]]:
    """
    Пример загрузки корпуса из .jsonl.

    Ожидаемый формат одной строки (пример):
      {
        "id": "doc_1",
        "title": "Название документа",
        "text": "Полный текст документа",
        ... (любые другие поля)
      }

    TODO: ПОДГОНИ ЭТУ ФУНКЦИЮ ПОД РЕАЛЬНЫЙ ФОРМАТ ДАННЫХ.
    Если корпус в .csv или в другой структуре — перепиши логику чтения.
    """
    docs: List[Dict[str, Any]] = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)
            docs.append(obj)
    return docs


# === Загружаем сырые документы ===
raw_docs = load_corpus_jsonl(CORPUS_PATH)

print(f"Загружено документов: {len(raw_docs)}")
print("Пример документа:")
print(raw_docs[0])


In [ ]:
# Готовим списки для build_corpus_from_texts
doc_texts = [d["text"] for d in raw_docs]
doc_ids = [str(d.get("id", i)) for i, d in enumerate(raw_docs)]
doc_titles = [d.get("title", "") for d in raw_docs]
doc_meta = [
    {k: v for k, v in d.items() if k not in ("id", "title", "text")}
    for d in raw_docs
]

# TODO: выбери чанкер для этой задачи
#   - paragraph_chunker: хорошо, если текст уже разбит на абзацы
#   - char_chunker: если текст с длинными абзацами/кодом
#   - token_chunker: если важен контроль по токенам (чуть сложнее)
CHUNKER = DEFAULT_CHUNKER

corpus, chunk_texts = build_corpus_from_texts(
    texts=doc_texts,
    doc_ids=doc_ids,
    titles=doc_titles,
    meta_list=doc_meta,
    chunker=CHUNKER,
)

print("=== Статистика корпуса ===")
print(f"Документов: {len(corpus.documents)}")
print(f"Чанков: {len(corpus.chunks)}")

# Покажем один пример чанка
example_chunk = corpus.chunks[0]
print("\nПример чанка:")
print(f"chunk_idx: {example_chunk.chunk_idx}")
print(f"doc_id:    {example_chunk.doc_id}")
print(f"start-end: {example_chunk.start_char}-{example_chunk.end_char}")
print("text:", example_chunk.text[:300].replace("\n", " ") + "...")


In [ ]:
# === BM25 индекс ===
# build_bm25_index ожидает список текстов чанков (и, возможно, мету).
# Мы уже получили chunk_texts из build_corpus_from_texts.

bm25_index: BM25Index = build_bm25_index(
    texts=chunk_texts,
    meta=None,  # можно передать список метаданных по чанкам, если нужно
)

print("BM25Index готов.")
print(f"Количество документов в BM25Index: {bm25_index.n_docs}")


In [ ]:
# === Эмбеддинг-модель ===
# TODO: подставь реальную модель эмбеддингов.
# Частый кейс: sentence-transformers (если разрешены в окружении олимпиады).

try:
    from sentence_transformers import SentenceTransformer

    EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # TODO: поменяй при необходимости
    print(f"Загружаем эмбеддинг-модель: {EMB_MODEL_NAME}")
    emb_model = SentenceTransformer(EMB_MODEL_NAME)
except ImportError:
    emb_model = None
    print("⚠️ sentence_transformers не установлен. "
          "Либо поставь пакет, либо подставь свою модель/обёртку.")

# === Dense индекс ===

if emb_model is not None:
    dense_index: DenseIndex = build_dense_index(
        texts=chunk_texts,
        emb_model=emb_model,
        batch_size=256,   # TODO: можно уменьшать/увеличивать в зависимости от VRAM
        cache_dir=os.path.join(DATA_DIR, "emb_cache"),  # кэш эмбеддингов
        model_id=EMB_MODEL_NAME,
    )
    print("DenseIndex готов.")
    print(f"Форма матрицы эмбеддингов: {dense_index.embeddings.shape}")
else:
    dense_index = None
    print("DenseIndex не построен (нет emb_model). "
          "Если нужен dense/hybrid — вернись и инициализируй модель.")


In [ ]:
def show_chunk(corpus: Corpus, chunk_idx: int, max_chars: int = 400) -> None:
    """
    Печатает один чанк с контекстом: doc_id, позиция, сам текст (обрезанный).
    """
    if chunk_idx < 0 or chunk_idx >= len(corpus.chunks):
        print(f"chunk_idx {chunk_idx} вне диапазона.")
        return

    ch = corpus.chunks[chunk_idx]
    doc = corpus.get_document(ch.doc_id)

    print("=" * 80)
    print(f"[chunk_idx={ch.chunk_idx}]  doc_id={ch.doc_id}")
    if doc is not None and getattr(doc, "title", None):
        print(f"title: {doc.title}")
    print(f"start-end: {ch.start_char}-{ch.end_char}")
    print("-" * 80)
    text = ch.text.replace("\n", " ")
    if len(text) > max_chars:
        text = text[:max_chars] + "..."
    print(text)
    print("=" * 80)


In [ ]:
def retrieve_bm25(
    query: str,
    bm25_index: BM25Index,
    k: int = TOP_K_DEBUG,
) -> List[Candidate]:
    return bm25_candidates(
        query=query,
        bm25_index=bm25_index,
        top_k=k,
    )


def retrieve_dense(
    query: str,
    dense_index: DenseIndex,
    emb_model,
    k: int = TOP_K_DEBUG,
) -> List[Candidate]:
    if dense_index is None or emb_model is None:
        raise ValueError("DenseIndex или emb_model не инициализированы.")
    return dense_candidates(
        query=query,
        dense_index=dense_index,
        emb_model=emb_model,
        top_k=k,
    )


def retrieve_hybrid(
    query: str,
    bm25_index: BM25Index,
    dense_index: Optional[DenseIndex],
    emb_model,
    k_bm25: int = 50,
    k_dense: int = 50,
    k_hybrid: int = TOP_K_DEBUG,
    alpha: float = 0.5,
) -> List[Candidate]:
    """
    Hybrid: alpha * bm25_score + (1 - alpha) * dense_score.

    k_bm25, k_dense — сколько кандидатов брать с каждой стороны,
    k_hybrid — сколько вернуть после объединения.
    """
    if dense_index is None or emb_model is None:
        raise ValueError("Для hybrid нужен и dense_index, и emb_model.")

    return hybrid_candidates(
        query=query,
        bm25_index=bm25_index,
        dense_index=dense_index,
        emb_model=emb_model,
        top_k_bm25=k_bm25,
        top_k_dense=k_dense,
        top_k=k_hybrid,
        alpha=alpha,
    )


In [ ]:
def print_candidates(
    name: str,
    cands: List[Candidate],
    corpus: Corpus,
    max_items: int = TOP_K_DEBUG,
    max_chars: int = 300,
) -> None:
    """
    Печатает список кандидатов: rank, score, doc_id, кусочек текста.
    """
    print(f"\n=== {name} (top {min(max_items, len(cands))}) ===")
    if not cands:
        print("Пусто.")
        return

    for c in cands[:max_items]:
        print(f"\n[r={c.rank:2d}] chunk_idx={c.chunk_idx:4d}  score={c.score:.4f}  source={c.source}")
        show_chunk(corpus, c.chunk_idx, max_chars=max_chars)


def compare_retrievers_for_query(
    query: str,
    bm25_index: BM25Index,
    dense_index: Optional[DenseIndex],
    emb_model,
    alpha: float = 0.5,
) -> None:
    """
    Запускает BM25 / Dense / Hybrid для одного запроса и печатает результат.
    """
    print("#" * 80)
    print(f"QUERY: {query}")
    print("#" * 80)

    bm25_cands = retrieve_bm25(query, bm25_index=bm25_index, k=TOP_K_DEBUG)
    print_candidates("BM25", bm25_cands, corpus)

    if dense_index is not None and emb_model is not None:
        dense_cands = retrieve_dense(query, dense_index=dense_index, emb_model=emb_model, k=TOP_K_DEBUG)
        print_candidates("Dense", dense_cands, corpus)

        hybrid_cands = retrieve_hybrid(
            query,
            bm25_index=bm25_index,
            dense_index=dense_index,
            emb_model=emb_model,
            k_bm25=50,
            k_dense=50,
            k_hybrid=TOP_K_DEBUG,
            alpha=alpha,
        )
        print_candidates(f"Hybrid (alpha={alpha})", hybrid_cands, corpus)
    else:
        print("\n⚠️ Dense/Hybrid пропущены (нет dense_index или emb_model).")


In [ ]:
# === Примеры запросов для отладки ===
# TODO: подставь реальные вопросы/запросы под конкретную задачу.
debug_queries = [
    "Что такое градиентный бустинг?",
    "Когда был запущен первый искусственный спутник Земли?",
    "Основные идеи метода преобразования Фурье",
]

for q in debug_queries:
    compare_retrievers_for_query(
        query=q,
        bm25_index=bm25_index,
        dense_index=dense_index,
        emb_model=emb_model,
        alpha=0.5,   # можно крутить, чтобы смотреть, как меняется выдача
    )


In [ ]:
def inspect_bm25_vs_hybrid_for_query(
    query: str,
    bm25_index: BM25Index,
    dense_index: Optional[DenseIndex],
    emb_model,
    alpha: float = 0.5,
    k_debug: int = 20,
):
    """
    Показывает табличку с кандидатами BM25 vs Hybrid — удобно для просмотра,
    какие чанки «поднимаются»/«падают» при переходе к hybrid.
    """
    bm25_cands = retrieve_bm25(query, bm25_index=bm25_index, k=k_debug)
    if dense_index is None or emb_model is None:
        print("Нет dense_index/emb_model, показываем только BM25.")
        df = summarize_candidates(bm25_cands, corpus)
        display(df.head(10))
        return

    hybrid_cands = retrieve_hybrid(
        query,
        bm25_index=bm25_index,
        dense_index=dense_index,
        emb_model=emb_model,
        k_bm25=k_debug,
        k_dense=k_debug,
        k_hybrid=k_debug,
        alpha=alpha,
    )

    df_before = summarize_candidates(bm25_cands, corpus)
    df_after = summarize_candidates(hybrid_cands, corpus)

    print("=== BM25 (до hybrid) ===")
    display(df_before.head(10))

    print("=== Hybrid (после объединения) ===")
    display(df_after.head(10))

    print("=== Анализ перехода (если нужен) ===")
    transition_df = analyze_stage_transition(
        before=bm25_cands,
        after=hybrid_cands,
        corpus=corpus,
    )
    display(transition_df.head(10))


# Пример использования:
inspect_bm25_vs_hybrid_for_query(
    query="Что такое градиентный бустинг?",
    bm25_index=bm25_index,
    dense_index=dense_index,
    emb_model=emb_model,
    alpha=0.5,
    k_debug=30,
)
